<a href="https://colab.research.google.com/github/aruaru0/espnet2-sample-japanese/blob/main/text_to_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text to Speech task utilizeing EPSnet2

## Installation

In [1]:
!pip install -q espnet pypinyin parallel_wavegan gdown espnet_model_zoo
!pip install -q --no-build-isolation pyopenjtalk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 24.3 MB/s eta 0:00:00
  Installing build depend

# text to speech

In [2]:
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.tts_inference import Text2Speech
from espnet2.utils.types import str_or_none
import torch

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [3]:
# モデル名などの設定(https://github.com/espnet/espnet_model_zoo/blob/master/espnet_model_zoo/table.csv　からjvs, justを選択)
tag = 'kan-bayashi/jsut_full_band_vits_prosody'
vocoder_tag = 'none'

In [4]:
text2speech = Text2Speech.from_pretrained(
    model_tag=str_or_none(tag),
    vocoder_tag=str_or_none(vocoder_tag),
    device="cuda",
)

https://zenodo.org/record/5521340/files/tts_train_full_band_vits_raw_phn_jaconv_pyopenjtalk_prosody_train.total_count.ave.zip?download=1: 100%|██████████| 357M/357M [11:50<00:00, 527kB/s]
/usr/local/lib/python3.10/dist-packages/espnet2/gan_tts/vits/monotonic_align/__init__.py:19: UserWarning: Cython version is not available. Fallback to 'EXPERIMETAL' numba version. If you want to use the cython version, please build it as follows: `cd espnet2/gan_tts/vits/monotonic_align; python setup.py build_ext --inplace`
  warnings.warn(
 - discriminator_params.follow_official_norm
 - discriminator_params.scale_discriminator_params.use_weight_norm
 - discriminator_params.scale_discriminator_params.use_spectral_norm

See also:
 - https://github.com/espnet/espnet/pull/5240
 - https://github.com/espnet/espnet/pull/5249


In [5]:
text = "吾輩は猫である。名前はまだ無い。　どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。"

with torch.no_grad():
    wav = text2speech(text)["wav"]

print(wav.shape[0]/text2speech.fs, "sec." )

Downloading: "https://github.com/r9y9/open_jtalk/releases/download/v1.11.1/open_jtalk_dic_utf_8-1.11.tar.gz"


dic.tar.gz: 0.00B [00:00, ?B/s]

Extracting tar file /usr/local/lib/python3.10/dist-packages/pyopenjtalk/dic.tar.gz
9.496961451247165 sec.


In [6]:
# 再生用のボタンを埋め込む
from IPython.display import display, Audio

display(Audio(wav.view(-1).cpu().numpy(), rate=text2speech.fs))

In [7]:
# wavデータとして保存する
import torchaudio

y = wav.reshape(1, len(wav)).cpu()
rate = text2speech.fs
torchaudio.save(filepath="test.wav", src=y, sample_rate=rate)
display(Audio("test.wav"))